In [ ]:
# Install streamlit
!pip install streamlit

In [ ]:
# Install ipykernel
!pip install -U ipykernel

In [ ]:
# Install opencv-python
!pip install opencv-python

In [ ]:
# Install pyngrok
!pip install pyngrok

In [ ]:
%%writefile app.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import requests
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from keras.utils.np_utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
import time
from sklearn.metrics import classification_report
import numpy as np
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
from io import BytesIO
import cv2
import random

# Streamit Web-app Configuration 
st.set_page_config(
     page_title="Image Classification App",
     page_icon="🧊",
     layout="wide",
     initial_sidebar_state="collapsed",
)
# Save the testing data from fashion-mnist_test.csv to the "test" DataFrame
test = pd.read_csv("drive/MyDrive/Colab Notebooks/fashion-mnist_test.csv")
# From the testing DataFrame, drop the 'label' attribute
X_test = test.drop(labels = ['label'], axis = 1)
# Save X_test to rand_test_img
rand_test_img = X_test
# Normalisation of our Dataframe (scale 0 to 1)
X_test = X_test / 255.0
# Reshape our DataFrame
X_test = X_test.values.reshape(-1,28,28,1)
# From the testing DataFrame, get the values of each label
Y_test = test['label'].values
# Convert to one-hot-encoding
Y_test = to_categorical(Y_test, num_classes = 10)

st.set_option('deprecation.showfileUploaderEncoding',False)
@st.cache(allow_output_mutation=True)
# With the following function, we load our saved model (you need to change it to your directory)
def load_model():
  model = tf.keras.models.load_model('drive/MyDrive/Colab Notebooks/my_final_model_4.hdf5')
  return model
# With the following function, we change the scale of the image given by the user of the application
def scale(image):
  image=tf.cast(image,tf.float32)
  image /=255.0
  return tf.image.resize(image,[28,28])
# With the following function, we decode the given image and expand its dimension
def decode_img(image):
  img = tf.image.decode_jpeg(image,channels=1)
  img=scale(img)
  return np.expand_dims(img, axis=0)


# We load the model
model = load_model()

# We create a list of our menu options
menu = ["Home", "Random Image Classification", "Upload & Classify", "Classify the URL", "About"]
# We create a list of our Classes
Cats = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
# We create a list with random facts for each class
facts_tshirt = ["The average price for a T-shirt/top is $19.86",
                "The first promo t-shirt was produced for The Wizard of Oz in 1939.", 
                "Coca-Cola did the first T-shirt brand promotion, especially in the 80s.", 
                "The world’s total cotton market is about $12 Billion.",
                "The world’s’ largest t-shirt is almost 300 feet long and also 200 feet wide."]
facts_trouser = ["In 2010, a judge in New York ruled that wearing sagging trousers is not necessarily a crime.",
                 "In Germany, it has been reported that dog bites leave 2,255 postmen with torn trousers every year.",
                 "The word ‘trousers’ was first recorded in English in 1625. Slightly earlier, the terms ‘trouse’, ‘trews’ or ‘strossers’ were occasionally seen.",
                 "“Never wear your best trousers when you go out to fight for freedom and truth” (Henrik Ibsen)."]
facts_pullover = ["Hoodie and Pullovers can be traced back to monks in Medieval age.",
                  "By the 1920s designers such as Jeanne Lanvin and Gabrielle (“Coco”) Chanel introduced sweaters into their collections.",
                  "Sweaters = clothing worn to produce sweating and reduce weight."]
facts_dress = ["Dresses were not originally a trend amongst women.",
               "In the 19th century, dresses took on a dramatic shape: the hoop-skirt and crinoline-supported styles were typically worn during the early 1800s.",
               "Michael Kors’ first project was his own mother’s wedding dress. He was a mere 5 years old at the time.",
               "Queen Victoria was the first person to wear a white wedding dress. Prior to this, white had been used as a color of mourning."]
facts_coat = ["The very first duffle coats were made for the British Navy.",
              "This garment is in fact representative of the war period.",
              "Jacket referred to a suit jacket, while coat referred to an overcoat or great coat."
              "Coat is one of the earliest clothing category words in English, attested as far back as the early Middle Ages."]
facts_sandal = ["Sandals were first worn by ancient Egyptians which were made up of papyrus and palm leaves.",
                "NASA sent an orthopedic pair of sandals called the Forceshoe into space.",
                "Flip Flops have been documented to be around since the Stone Ages through the discovery of cave paintings which depict people wearing sandals similar to flip flops.",
                "People in India used wood for their flip-flops."]
facts_shirt = ["Two billion shirts are made in one year.",
               "A shirt is a cloth garment for the upper body (from the neck to the waist).",
               "In the 1920s and 1930s, fascists wore different coloured shirts.",
               "The first shirt is 5000 years old"]
facts_sneaker = ["Michael Jordan wanted to sign with Adidas in 1984, not Nike. He was a self-described “Adidas nut,” and told his agent that if the deal was close he’d sign with them.",
                 "Sneakers got their name because their rubber soles do not make noise.",
                 "The big sneaker companies such as Nike and Adidas own viewing boxes at most large sporting events like the U.S. Open and NBA Finals. However, should you step into their box wearing the sneakers of a competitor, you may be promptly asked to go barefoot – or offered to exchange your shoes temporarily for the host brand’s.",
                 "The Nike Air Force I is one of the best-selling sneakers of all time and it was never advertised."]
facts_bag = ["Only since the nineteenth century does the handbag exist. In the days before, women wore boxes around their necks in the shape of a chain.",
             "A feng shui saying goes like this: 'A purse on the floor is money out the door.'",
             "The top five items in the handbag are: keys, wallet, telephone, handkerchiefs and pens.",
             "On average a woman spends 77 days of her life searching in her bag.",
             "The average number of items in a women's bag is 6.",
             "The Bible makes Judas Iscariot the first ever person to own a bag in history."]
facts_ankleboot = ["A type of soft leather ankle boots were worn by nomads in eastern Asia and carried to China to India and Russia around AD 1200 to 1500 by Mongol invaders",
                   "Ankle boots featuring chunky shapes, brassy hardware, and bold buckle details are especially cool in 2020",
                   "It was more common for women of the fourteenth century to wear laced ankle boots, which were often lined in fur.",
                   "Ankle boots—or booties—are arguably the ultimate fall shoe."]

# Define a dictionary
facts = {
    "T-shirt/top": facts_tshirt,
    "Trouser": facts_trouser,
    "Pullover": facts_pullover,
    "Dress": facts_dress,
    "Coat": facts_coat,
    "Sandal": facts_sandal,
    "Shirt": facts_shirt,
    "Sneaker": facts_sneaker,
    "Bag": facts_bag,
    "Ankle boot": facts_ankleboot
}

# Create a sidebar and add App Logo/Name
st.sidebar.image("drive/MyDrive/University-of-Piraeus-Logo.png")
st.sidebar.header("University Of Piraeus")
st.sidebar.write("Dept. of Digital Systems")
# Define the menu in the sidebar
choice = st.sidebar.selectbox("Menu", menu)
# Selected Option 'Home';
if choice == "Home":
    col1,col2,col3,col4 = st.beta_columns(4)
    st.header("**Image Classification App based on Fashion-MNIST dataset**")
    st.subheader("**Welcome!**")
    st.write("A quick peek to the Fashion-MNIST dataset")
    st.markdown("![Alt Text](https://i.imgur.com/IUKrXTa.gif)")
    st.write("This web app can classify clothing types into 10 Fashion Categories with the following options;")
    st.write("◘ Random Image Classification, ")
    st.write("◘ Upload & Classify, and")
    st.write("◘ Classify the URL")
    st.write("Head over to the menu section to check them out;)")

# Selected Option 'Random Image Classification';
if choice == "Random Image Classification":
    st.header("**Fashion Image Classification App**")
    st.subheader("In this section, you can see the prediction of a random image from the test dataset.")
    st.write("By clicking the 'Predict new image' button, the prediction of a new image is presented and two random facts about the predicted item.")
    col1,col2,col3,col4,col5,col6 = st.beta_columns(6)
    if col4.button("Predict new image"):
        # Get a random image from the testing dataset
        random_img = rand_test_img.sample(n=1)
        # Normalisation of the random image
        random_img = random_img / 255.0
        # Reshape the image
        random_img = random_img.values.reshape(-1,28,28,1)
        # Show image to user
        st.image(random_img,width = 400,use_column_width=False)
        a = np.expand_dims(random_img, axis = 3)
        # Predict from model
        prediction=model.predict_classes(a)
        # Get the prediction percentage matrix
        prediction_percentages = model.predict(a)
        # Get the maximum prediction percentage 
        max_index = prediction_percentages.max()   
        # Get label from indexes  
        pred_label=Cats[int(prediction[0])]
        # Print the result of the prediction in our web-app
        st.subheader("Randomly-selected Image is a "+str(pred_label)+" with "+str(max_index*100)+"% .")
        # For different prediction percentages, plot different reaction emojis
        if max_index > 0.9:
            col_1, col_2 = st.beta_columns([1,8]) 
            with col_1:
                st.write('Prediction reaction: ') 
            with col_2:
                st.image("drive/MyDrive/robot.png", width = 30)
        elif max_index > 0.8:
            col_1, col_2 = st.beta_columns([1,8]) 
            with col_1:
                st.write('Prediction reaction: ') 
            with col_2:
                st.image("drive/MyDrive/smiling-face-with-sunglasses.png", width = 30)
        elif max_index > 0.7:
            col_1, col_2 = st.beta_columns([1,8]) 
            with col_1:
                st.write('Prediction reaction: ') 
            with col_2:
                st.image("drive/MyDrive/face-with-monocle.png", width = 30)
        elif max_index > 0.5:
            col_1, col_2 = st.beta_columns([1,8]) 
            with col_1:
                st.write('Prediction reaction: ') 
            with col_2:
                st.image("drive/MyDrive/neutral-face.png", width = 30)
        else:
            col_1, col_2 = st.beta_columns([1,8]) 
            with col_1:
                st.write('Prediction reaction: ') 
            with col_2:
                st.image("drive/MyDrive/lying-face.png", width = 30)
        # Print random facts
        st.write("Random facts: ")
        rand_fact = random.sample(facts[pred_label],2)
        st.write("1. ",rand_fact[0])
        st.write("2. ",rand_fact[1])

# Selected Option 'Upload & Classify';
if choice == "Upload & Classify":
    st.header("**Fashion Image Classification App**")
    st.subheader("In this section, you can see the prediction of your uploaded image.")
    # Save locally the image that the user uploaded
    file=st.file_uploader("Please upload clothing image file.", type = ["png","jpg"])
    # If the file exists;
    if file:
        # Read image as array
        file_bytes = np.asarray(bytearray(file.read()), dtype=np.uint8)
        # Convert array to image
        opencv_image = cv2.imdecode(file_bytes, 1)
        # Convert RGB Image to a grayscale
        gray = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2GRAY)
        # Show image to user
        st.image(opencv_image,width=600,use_column_width=False)
        # Original Fashion Dataset pixels
        dim =(28,28) 
        # Resize image to 28*28
        new_img = cv2.resize(gray,dim)
        # Add dimension to 0 axis
        arr = np.expand_dims(new_img, axis=0)
        # Add dimension to 3 axis
        images = np.expand_dims(arr, axis=3)
        # Predict from model
        prediction=model.predict_classes(images)
        # Get label from indexes
        pred_label=Cats[int(prediction[0])]
        # Print the result of the prediction
        st.subheader("Uploaded Image is a "+str(pred_label)+".")
        st.write("Random facts: ")
        # Get a random fact
        rand_fact = random.sample(facts[pred_label],2)
        st.write("1. ",rand_fact[0])
        st.write("2. ",rand_fact[1])

# Selected Option 'Classify the URL';  
if choice == "Classify the URL":
    st.header("**Fashion Image Classification App**")
    st.subheader("In this section, you can see the prediction of an image, by providing a URL.")
    # Save the link that the user provides
    path = st.text_input("Enter image URL :", "http://pngimg.com/uploads/women_bag/women_bag_PNG6428.png")
    # If the URL is True;
    if path:
        # Load the image from the link provided by the user of the web-app
        content = requests.get(path).content
        image = Image.open(BytesIO(content))
        # Show the image to the user of the web-app
        st.image(image,width = 600, use_column_width=False)
        # Spin until the prediction results are ready
        with st.spinner("classifying.."):
            # Predict the decoded image
            label = np.argmax(model.predict(decode_img(content)), axis = 1)
            # Get the label based on the prediction results
            pred_label=Cats[label[0]]
            # Print the prediction results to the user
            st.subheader("The image from the URL is a "+str(pred_label)+".")
            st.write("Random facts: ")
            # Retrieve two random fact
            rand_fact = random.sample(facts[pred_label],2)
            # Show the random facts
            st.write("1. ",rand_fact[0])
            st.write("2. ",rand_fact[1])

# Selected Option 'About';  
if choice == "About":
    # Print general/contact information about the creators of the web-app
    st.subheader("About")
    st.write("This application was designed as part of the 'Advanced Data Analysis Topics' course, by Stefanatou Gerasimina and Panagiotis Ioannis Dallas under the supervision of Halkidi Maria, Associate Professor of the Digital Systems dept. at the University of Piraeus!")
    
    st.write("For more information about the Fashion-MNIST dataset you can visit Kaggle on https://www.kaggle.com/zalando-research/fashionmnist")
    
    st.subheader("Contact")
    st.write("For questions, bugs or suggestions feel free to contact us by mail stefanatougerasimina@gmail.com or panagiotisdallasunipi@gmail.com")


Overwriting app.py


In [ ]:
#Attach the file output 
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
# Initialize connection with localhost
from pyngrok import ngrok
options = {
   "port": 8501
}
url=ngrok.connect(options=options)
url

<NgrokTunnel: "http://e1ee92de0994.ngrok.io" -> "http://localhost:80">

In [ ]:
# Run streamlit server at 80 port
# Click the generated link (http://XXXXX.ngrok.io)of the NgrokTunnel above, to open the web-application
!streamlit run --server.port 80 --server.enableXsrfProtection=false app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://35.245.214.97:80

2021-02-20 22:31:20.774137: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-02-20 22:31:26.995 NumExpr defaulting to 2 threads.
2021-02-20 22:31:27.236441: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-20 22:31:27.237303: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-20 22:31:27.266532: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-02-20 22:31:27.267147: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCap